# 词嵌入(Embedding)技术详解

## 一、什么是词嵌入?

词嵌入(Word Embedding)是自然语言处理(NLP)中的一项核心技术，它将文本中的词语转换为稠密的数值向量表示。

### 为什么需要词嵌入？
- **计算机无法直接理解文字**：计算机只能处理数字，需要将文字转换为数值形式
- **独热编码的局限性**：传统的独热编码(One-hot)表示维度过高且无法表达词语之间的关系
- **语义表示**：词嵌入可以将语义相近的词映射到向量空间中相近的位置

### 词嵌入的核心思想
将每个词映射为一个固定长度的实数向量（如5维、100维、300维等），这个向量能够捕捉词语的语义信息。例如：
- "国王" - "男人" + "女人" ≈ "女王"
- 相似的词（如"猫"和"狗"）在向量空间中距离较近

## 二、本示例的学习目标

1. 理解中文文本的分词处理流程
2. 掌握词表(Vocabulary)的构建方法
3. 学习PyTorch中Embedding层的使用
4. 理解词向量的表示和意义

---

In [11]:
# ============================================
# 导入必要的库
# ============================================

import torch              # PyTorch核心库，用于张量计算
import torch.nn as nn     # PyTorch神经网络模块，提供Embedding层等
import jieba              # 结巴分词库，用于中文分词

# jieba分词原理：
# 1. 基于前缀词典实现高效的词图扫描
# 2. 生成句子中汉字所有可能成词情况所构成的有向无环图(DAG)
# 3. 采用动态规划查找最大概率路径，找出基于词频的最大切分组合
# 4. 对于未登录词，采用HMM模型进行识别

## 步骤1：准备原始文本数据

这是一段关于自然语言处理的中文描述，我们将用它来演示词嵌入的完整流程。

In [12]:
# ============================================
# 定义原始文本
# ============================================

# 这是待处理的原始中文文本
# 注意：中文不像英文那样天然用空格分隔单词，需要进行分词处理
text = "自然语言是由文字构成的，而语言的含义是由单词构成的。即单词是含义的最小单位。因此为了让计算机理解自然语言，首先要让它理解单词含义。"

## 步骤2：使用jieba进行中文分词

### jieba.lcut()函数详解

**函数原型**：`jieba.lcut(sentence, cut_all=False, HMM=True)`

**参数说明**：
- `sentence`：需要分词的字符串
- `cut_all`：是否使用全模式（默认False，精确模式）
- `HMM`：是否使用HMM模型识别未登录词（默认True）

**返回值**：返回一个列表(list)，包含分词后的所有词语

### 分词的重要性
中文句子是连续的字符序列，不像英文有明确的空格分隔。分词是中文NLP的第一步，将连续的文本切分成有意义的词语单元。

In [13]:
# ============================================
# 步骤1：中文分词
# ============================================

# 使用jieba的lcut方法对文本进行分词
# lcut = list cut，返回分词结果的列表
# 例如："自然语言" 会被识别为一个完整的词，而不是"自然"和"语言"两个词
original_words = jieba.lcut(text)

# 打印分词结果，可以看到文本被切分成了一个个独立的词语
# 结果包含了实词（如"自然语言"、"计算机"）和虚词（如"是"、"的"）
print(original_words)

['自然语言', '是', '由', '文字', '构成', '的', '，', '而', '语言', '的', '含义', '是', '由', '单词', '构成', '的', '。', '即', '单词', '是', '含义', '的', '最小', '单位', '。', '因此', '为了', '让', '计算机', '理解', '自然语言', '，', '首先', '要', '让', '它', '理解', '单词', '含义', '。']


## 步骤3：定义停用词表

### 什么是停用词？

停用词(Stop Words)是指在文本处理中需要过滤掉的词语，这些词通常包括：
- **高频虚词**："的"、"是"、"在"等，出现频率高但语义信息少
- **标点符号**："，"、"。"、"！"等
- **语气助词**："啊"、"呢"、"吗"等

### 为什么要过滤停用词？

1. **降低维度**：减少词表大小，降低模型复杂度
2. **提升效率**：减少计算量和存储空间
3. **突出重点**：保留更有意义的实词，提高模型对关键信息的关注
4. **提高准确性**：去除噪声词，让模型更专注于有意义的内容

在实际项目中，通常会使用更完整的停用词表（如哈工大停用词表、百度停用词表等）。

In [14]:
# ============================================
# 步骤2：定义停用词表
# ============================================

# 停用词是在文本处理时需要过滤掉的词，通常是高频但语义信息量少的词
# 使用集合(set)数据结构，因为集合的查找效率是O(1)，比列表的O(n)更高效
stopwords = {"的", "是", "而", "由", "，", "。"}

# 注意：这里只定义了少量停用词作为示例
# 在真实项目中，停用词表通常包含数百个词，需要从文件中加载

## 步骤4：过滤停用词

使用列表推导式(List Comprehension)快速过滤掉停用词，只保留有意义的实词。

**代码解析**：`[word for word in original_words if word not in stopwords]`
- 遍历原始分词结果中的每个词
- 检查该词是否在停用词表中
- 如果不在停用词表中，则保留该词

过滤后的结果将更加简洁，只包含有实际语义价值的词语。

In [15]:
# ============================================
# 步骤3：过滤停用词
# ============================================

# 使用列表推导式过滤停用词
# 语法：[表达式 for 变量 in 可迭代对象 if 条件]
# 这里的条件是 word not in stopwords，即词不在停用词集合中
words = [word for word in original_words if word not in stopwords]

# 打印过滤后的词列表
# 对比之前的结果，可以看到"的"、"是"、"，"、"。"等停用词已被移除
# 现在只剩下"自然语言"、"文字"、"构成"等有实际意义的词
print(words)


['自然语言', '文字', '构成', '语言', '含义', '单词', '构成', '即', '单词', '含义', '最小', '单位', '因此', '为了', '让', '计算机', '理解', '自然语言', '首先', '要', '让', '它', '理解', '单词', '含义']


## 步骤5：构建词表(Vocabulary) - id2word映射

### 什么是词表？

词表(Vocabulary)是模型中所有不重复词语的集合，是连接文本和数字的桥梁。

### id2word的含义

- **id2word**：索引到词的映射（列表形式）
- 列表的下标就是词的ID
- 列表的值就是对应的词语
- 例如：`id2word[0]` 可能返回 "自然语言"

### 构建过程

1. `set(words)`：将词列表转换为集合，**自动去重**
2. `list(...)`：将集合转回列表，方便通过索引访问

**注意**：集合是无序的，所以每次运行得到的词表顺序可能不同。在实际项目中，通常会对词表排序以保证一致性。

In [16]:
# ============================================
# 步骤4：构建词表（id2word映射）
# ============================================

# 词表构建的两个步骤：
# 1. set(words): 将词列表转为集合，自动去除重复的词
#    例如：["单词", "含义", "单词"] -> {"单词", "含义"}
# 2. list(...): 将集合转回列表，这样可以通过索引访问
#    列表的索引(0, 1, 2, ...)就是词的ID
id2word = list(set(words))

# 打印词表
# id2word是一个列表，存储了所有不重复的词
# 例如：id2word[0]表示ID为0的词，id2word[1]表示ID为1的词
# 这个词表的大小(长度)就是词汇表的规模
print(id2word)

['自然语言', '最小', '计算机', '首先', '为了', '文字', '单位', '它', '构成', '含义', '让', '理解', '要', '单词', '因此', '即', '语言']


## 步骤6：构建反向映射 - word2id字典

### word2id的作用

- **word2id**：词到索引的映射（字典形式）
- 与id2word互为反向映射
- 用于快速查找某个词对应的ID
- 例如：`word2id["自然语言"]` 可能返回 `0`

### 为什么需要两种映射？

- **id2word（列表）**：适合通过ID查词，用于解码/输出阶段
- **word2id（字典）**：适合通过词查ID，用于编码/输入阶段

### 构建方法

使用字典推导式：`{word: i for i, word in enumerate(id2word)}`
- `enumerate(id2word)`：同时获取索引i和词word
- 构建字典时，词作为键(key)，索引作为值(value)

这两个映射在NLP任务中经常同时使用，构成完整的词汇索引系统。

In [17]:
# ============================================
# 步骤5：构建反向映射（word2id字典）
# ============================================

# 使用字典推导式构建word2id映射
# enumerate(id2word) 会返回 (索引, 词语) 的元组
# 例如：enumerate(["苹果", "香蕉"]) -> (0, "苹果"), (1, "香蕉")
# 字典推导式语法：{键表达式: 值表达式 for 变量 in 可迭代对象}
word2id = {word: i for i, word in enumerate(id2word)}

# 打印word2id字典
# 这是一个字典，键是词语，值是对应的ID
# 例如：{"自然语言": 0, "计算机": 2, "理解": 11, ...}
# 当我们需要把词转换为ID时，直接使用 word2id["某个词"] 即可
print(word2id)

{'自然语言': 0, '最小': 1, '计算机': 2, '首先': 3, '为了': 4, '文字': 5, '单位': 6, '它': 7, '构成': 8, '含义': 9, '让': 10, '理解': 11, '要': 12, '单词': 13, '因此': 14, '即': 15, '语言': 16}


## 步骤7：创建词嵌入层(Embedding Layer)

### nn.Embedding详解

`nn.Embedding` 是PyTorch提供的词嵌入层，本质上是一个**查找表(Lookup Table)**。

### 参数说明

```python
nn.Embedding(num_embeddings, embedding_dim)
```

- **num_embeddings**：词汇表的大小（有多少个不同的词）
  - 这里是 `len(id2word)`，即词表中词的数量
  - 例如：如果词表有17个词，则 num_embeddings=17

- **embedding_dim**：词向量的维度（用多少个数字表示一个词）
  - 这里设置为5，表示用5个实数表示一个词
  - 实际应用中常用50、100、300等维度
  - 维度越高，表达能力越强，但计算成本也越高

### 内部结构

Embedding层内部维护一个形状为 `(num_embeddings, embedding_dim)` 的权重矩阵：
- 每一行代表一个词的向量表示
- 例如：第0行是ID为0的词的向量，第1行是ID为1的词的向量
- 初始时，这些向量是**随机初始化**的
- 在训练过程中，这些向量会被不断优化，学习到词的语义信息

### 工作原理

当输入一个词的ID时，Embedding层会：
1. 在权重矩阵中查找对应行
2. 返回该行的向量作为词的表示

这个过程非常高效，时间复杂度是O(1)。

In [18]:
# ============================================
# 步骤6：创建词嵌入层
# ============================================

# nn.Embedding是PyTorch中的词嵌入层
# 它本质上是一个查找表，存储了每个词对应的向量

# 参数1：num_embeddings - 词汇表大小（有多少个不同的词）
#       这里是len(id2word)，即词表中有多少个词
# 参数2：embedding_dim - 词向量的维度（用多少个数字表示一个词）
#       这里设置为5，表示每个词用5个实数来表示
#       实际应用中常用50、100、300等维度

embed = nn.Embedding(num_embeddings=len(id2word), embedding_dim=5)

# Embedding层内部维护一个形状为(num_embeddings, embedding_dim)的权重矩阵
# 本例中是(17, 5)，即17个词，每个词用5维向量表示
# 这些向量初始是随机的，在实际训练中会不断更新优化

## 步骤8：使用Embedding层进行前向传播

### 前向传播过程

遍历词表中的每个词，获取其对应的词向量并打印。

### 代码详解

1. **enumerate(id2word)**：遍历词表，同时获取索引和词语

2. **torch.tensor(id)**：将Python整数转换为PyTorch张量
   - Embedding层的输入必须是张量类型
   - 这个张量包含了词的ID

3. **embed(torch.tensor(id))**：Embedding层的前向传播
   - 输入：词的ID（整数张量）
   - 输出：对应的词向量（浮点数张量）
   - 这个过程就是在权重矩阵中查找对应行

4. **word_vec.detach().numpy()**：转换为NumPy数组便于打印
   - `detach()`：从计算图中分离，不再追踪梯度
   - `numpy()`：转换为NumPy数组格式

### 输出结果解读

输出格式：`ID: 词语    [向量值]`

例如：`0: 自然语言  [1.334826, -0.8659162, -0.5738347, -1.7312728, 0.9658297]`

- **ID为0**：表示"自然语言"在词表中的索引是0
- **5个实数**：这就是"自然语言"的向量表示
- 这些数值目前是随机的，还没有经过训练
- 在实际应用中，经过训练后，语义相近的词向量会比较接近

### 词向量的意义

- 每个词都被映射到一个5维空间中的点
- 向量的每个维度可以理解为词的某种"特征"
- 通过训练，模型会学习到有意义的特征表示
- 例如：某个维度可能表示"具体-抽象"，另一个维度可能表示"积极-消极"

In [19]:
# ============================================
# 步骤7：前向传播 - 获取每个词的词向量
# ============================================

# 遍历词表中的每个词，获取其对应的词向量
for id, word in enumerate(id2word):
    # 1. torch.tensor(id): 将词的ID转换为PyTorch张量
    #    Embedding层的输入必须是张量，不能是普通的Python整数
    
    # 2. embed(...): 调用Embedding层进行前向传播
    #    输入是词的ID，输出是对应的词向量
    #    这个过程就是在embedding矩阵中查找第id行
    word_vec = embed(torch.tensor(id))
    
    # 3. detach(): 从计算图中分离，不再追踪梯度（因为这里只是查看，不需要反向传播）
    # 4. numpy(): 转换为NumPy数组，方便打印查看
    
    # 5. 格式化输出：
    #    {id:>2} - ID右对齐，占2个字符宽度
    #    {word:8} - 词语左对齐，占8个字符宽度（中文占2个字符）
    #    word_vec.detach().numpy() - 打印词向量的数值
    print(f"{id:>2}: {word:8}\t {word_vec.detach().numpy()}")

# 输出解读：
# 每行显示：词的ID、词本身、词向量（5个实数）
# 例如：0: 自然语言  [1.33, -0.87, -0.57, -1.73, 0.97]
# 这5个数字就是"自然语言"这个词在向量空间中的坐标
# 注意：这些向量是随机初始化的，还未经过训练优化

 0: 自然语言    	 [ 0.53865093 -1.0054226  -0.0523538  -0.76718557  0.28203788]
 1: 最小      	 [0.11028367 1.0005876  1.5475931  0.92961705 0.468658  ]
 2: 计算机     	 [-1.1540222   0.12683915  0.11135134 -0.3644273   0.76995766]
 3: 首先      	 [ 0.06972941 -1.4217769  -0.0301713   0.3017562   1.8669455 ]
 4: 为了      	 [ 1.6309752   1.5363684  -0.9255767  -0.06418242 -1.5632848 ]
 5: 文字      	 [ 1.2455223  0.6569212  1.0844438 -1.1405432  0.8340077]
 6: 单位      	 [-0.64971966  0.8060929  -1.6751878  -0.804138   -0.15776595]
 7: 它       	 [1.100047   1.1607912  0.24624968 0.64584327 1.4690481 ]
 8: 构成      	 [ 0.10094528 -0.28397772  0.29423493 -0.13099904 -0.7861144 ]
 9: 含义      	 [-0.9374758   0.04340686 -0.65925616 -2.4856915   1.9722801 ]
10: 让       	 [ 0.7704642  -1.3223021   0.06516912 -0.6066528  -0.6808615 ]
11: 理解      	 [ 0.70741206  0.15819274  0.40426585 -2.017006   -0.22190902]
12: 要       	 [ 0.03838061  0.5149802  -0.40113607 -0.32069087  0.50942624]
13: 单词      	 [ 0.3088705  

---

## 总结：词嵌入的完整流程

### 处理流程回顾

1. **文本分词**：使用jieba将连续的中文文本切分成词语
2. **过滤停用词**：去除高频但语义信息少的词，降低噪声
3. **构建词表**：创建id2word和word2id两种映射关系
4. **创建Embedding层**：初始化词嵌入查找表
5. **获取词向量**：通过词ID查找对应的向量表示

### 关键技术点

| 技术 | 作用 | 实现方式 |
|------|------|----------|
| 分词 | 将文本切分成词语单元 | jieba.lcut() |
| 停用词过滤 | 去除无意义的高频词 | 集合查找 |
| 词表构建 | 建立词与ID的双向映射 | 集合去重 + 字典推导 |
| Embedding层 | 将词ID映射为稠密向量 | nn.Embedding() |
| 前向传播 | 查找词对应的向量 | embed(tensor) |

### 实际应用场景

词嵌入是许多NLP任务的基础，包括：
- **文本分类**：情感分析、主题分类等
- **序列标注**：命名实体识别、词性标注等
- **机器翻译**：将源语言翻译为目标语言
- **问答系统**：理解问题并生成答案
- **文本生成**：自动写作、对话系统等

### 进阶学习方向

1. **预训练词向量**：Word2Vec、GloVe、FastText
2. **上下文相关嵌入**：ELMo、BERT、GPT
3. **子词嵌入**：Byte Pair Encoding (BPE)、WordPiece
4. **多语言嵌入**：跨语言词向量对齐

### 注意事项

- 本示例中的词向量是**随机初始化**的，还未经过训练
- 实际应用中，需要在具体任务上训练，让词向量学习到有意义的语义表示
- 也可以使用预训练的词向量（如Word2Vec、GloVe），这些向量已经在大规模语料上训练好
- 词表大小和向量维度的选择需要在表达能力和计算效率之间权衡